In [1]:
#Imports
import pandas as pd
from pathlib import Path
import os
import numpy as np


In [2]:
projectRoot = Path.cwd().parent.parent
TRACK_DIR   = projectRoot / "cleanedData" / "train"
WEEK_FMT  = "oriented_2023_w{week:02d}.csv"
print(TRACK_DIR)

/Users/luisbapst/Code/BDB26/cleanedData/train


In [3]:
#Read in Data
projectRoot = Path.cwd().parent
dfPlays = pd.read_csv(f'{projectRoot}/selectedPlays.csv')

In [4]:
to_num = lambda s: pd.to_numeric(s, errors="coerce")

In [5]:
def load_week(week) -> pd.DataFrame:
    w = int(week)
    f = TRACK_DIR / WEEK_FMT.format(week=w)              # w01..w09
    if not f.exists():
        # fallback: try without zero-padding just in case (w1..w9)
        f2 = TRACK_DIR / f"oriented_2023_w{w}.csv"
        if not f2.exists():
            print(f"[load_week] Missing file for week {w:02d}: {f.name}")
            return pd.DataFrame()
        f = f2
    return pd.read_csv(f)

In [6]:
def get_throw_frame_last(trk_play: pd.DataFrame) -> int | None:
    # Throw is the last frame in your tracking
    return int(trk_play["frame_id"].max()) if len(trk_play) else None

In [7]:
def mask_qb(df: pd.DataFrame) -> pd.Series:
    # prefer role 'Passer', fallback to position 'QB'
    role = df["player_role"].astype(str).str.lower()
    pos  = df["player_position"].astype(str).str.upper()
    return role.eq("passer") | pos.eq("QB")

In [8]:
def mask_targeted(df: pd.DataFrame) -> pd.Series:
    role = df["player_role"].astype(str).str.lower()
    return role.eq("targeted receiver")

In [9]:
def mask_defense(df: pd.DataFrame) -> pd.Series:
    return df["player_side"].astype(str).str.lower().eq("defense")

In [10]:
def dist_xy_row(a: pd.Series, b: pd.Series) -> float:
    return float(np.hypot(a["x"] - b["x"], a["y"] - b["y"]))

In [11]:
def nearest_defender_sep(frame0: pd.DataFrame, wr_row: pd.Series) -> float:
    defs = frame0[mask_defense(frame0)]
    if defs.empty: return np.nan
    dists = defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1)
    return float(dists.min())

In [12]:
def density_within(frame0: pd.DataFrame, wr_row: pd.Series, radius: float) -> int:
    defs = frame0[mask_defense(frame0)]
    if defs.empty: return 0
    return int((defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1) <= radius).sum())

In [13]:
def nearest_defender(frame0: pd.DataFrame, wr_row: pd.Series):
    defs = frame0[mask_defense(frame0)]
    if defs.empty: 
        return np.nan, None
    dists = defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1)
    j = dists.idxmin()
    return float(dists.loc[j]), defs.loc[j]

In [14]:
def _vel_components(row: pd.Series):
    if "s" not in row or "dir" not in row:
        return np.nan, np.nan
    s   = float(pd.to_numeric(row["s"], errors="coerce"))
    ang = float(pd.to_numeric(row["dir"], errors="coerce"))
    if not np.isfinite(s) or not np.isfinite(ang):
        return np.nan, np.nan
    rad = np.deg2rad(ang)
    return s*np.cos(rad), s*np.sin(rad)

In [ ]:
def closing_speed_defender(wr_row: pd.Series, def_row: pd.Series) -> float:
    # unit vector from DEF -> WR
    dx = float(wr_row["x"] - def_row["x"])
    dy = float(wr_row["y"] - def_row["y"])
    dist = float(np.hypot(dx, dy))
    if dist == 0:
        return np.nan
    ux, uy = dx/dist, dy/dist

    vwx, vwy = _vel_components(wr_row)
    vdx, vdy = _vel_components(def_row)
    if not (np.isfinite(vwx) and np.isfinite(vwy) and np.isfinite(vdx) and np.isfinite(vdy)):
        return np.nan

    return (vdx - vwx)*ux + (vdy - vwy)*uy

In [ ]:
# helper (put once, e.g., above your feature function)
def encode_man_zone(val) -> float:
    if pd.isna(val):
        return np.nan
    v = str(val).strip().upper()
    if v == "ZONE_COVERAGE":
        return 0.0
    if v == "MAN_COVERAGE":
        return 1.0
    # unknown/mixed -> NaN (you can choose 0.5 if you prefer)
    return np.nan


In [ ]:
ROUTE_VOCAB = [
    "IN","POST","OUT","CORNER","HITCH","CROSS","FLAT","GO",
    "SLANT","ANGLE","SCREEN","WHEEL","UNKNOWN"
]

def one_hot_route(route_val) -> dict:
    import math
    r = str(route_val).strip().upper() if route_val is not None and not (isinstance(route_val, float) and math.isnan(route_val)) else "UNKNOWN"
    if r not in ROUTE_VOCAB:
        r = "UNKNOWN"
    # build dense one-hots as floats
    return {f"route_{name.lower()}": (1.0 if name == r else 0.0) for name in ROUTE_VOCAB}


In [ ]:
def features_at_throw_lastframe(frame0: pd.DataFrame, play_row: pd.Series) -> dict | None:
    qb = frame0[mask_qb(frame0)].head(1)
    receiver = frame0[mask_targeted(frame0)].head(1)
    if receiver.empty:
        off_mask = frame0["player_side"].astype(str).str.lower().eq("offense")
        receiver = frame0[off_mask & (~mask_qb(frame0))].head(1)
    if qb.empty or receiver.empty:
        return None

    qb, receiver = qb.iloc[0], receiver.iloc[0]

    # nearest defender + speed + closing
    sep0, def_row = nearest_defender(frame0, receiver)
    def1_speed   = float(pd.to_numeric(def_row.get("s", np.nan), errors="coerce")) if def_row is not None else np.nan
    def1_closing = closing_speed_defender(receiver, def_row) if def_row is not None else np.nan

    # receiver speed
    receiver_speed = float(pd.to_numeric(frame0.loc[receiver.name, "s"], errors="coerce")) if "s" in frame0.columns else np.nan

    # target (ball landing) -> relative distances ONLY
    if ("ball_land_x" not in frame0.columns) or ("ball_land_y" not in frame0.columns):
        return None
    tx = float(pd.to_numeric(frame0["ball_land_x"].iloc[0], errors="coerce"))
    ty = float(pd.to_numeric(frame0["ball_land_y"].iloc[0], errors="coerce"))
    if not (np.isfinite(tx) and np.isfinite(ty)):
        return None

    # in-bounds check AFTER finiteness
    in_bounds = (0.0 < ty < 53.3) and (tx < 110.0)
    if not in_bounds:
        return None

    # coerce coords to float before hypot
    qx, qy = float(qb["x"]), float(qb["y"])
    rx, ry = float(receiver["x"]), float(receiver["y"])

    #route OH
    route_oh = one_hot_route(play_row.get("route_of_targeted_receiver"))

    target_depth       = float(np.hypot(tx - qx, ty - qy))
    receiver_to_target = float(np.hypot(tx - rx, ty - ry))

    if def_row is not None and np.isfinite(def_row["x"]) and np.isfinite(def_row["y"]):
        dx, dy = float(def_row["x"]), float(def_row["y"])
        def1_to_target = float(np.hypot(tx - dx, ty - dy))
    else:
        def1_to_target = np.nan

    return {
        "game_id":                 int(play_row["game_id"]),
        "play_id":                 int(play_row["play_id"]),
        "week":                    int(play_row["week"]),
        # at-throw features
        "receiver_sep0_yd":        float(sep0) if np.isfinite(sep0) else np.nan,
        "def1_speed0_mps":         def1_speed,
        "def1_closing_mps":        def1_closing,  
        "receiver_speed0_mps":     receiver_speed,
        "eta_rec"   :   receiver_to_target / max(receiver_speed, 0.1),
        "eta_def" :   def1_to_target       / max(def1_speed, 0.1),
        "def_density_r20":         density_within(frame0, receiver, 2.0),
        "def_density_r30":         density_within(frame0, receiver, 3.0),
        "def_density_r50":         density_within(frame0, receiver, 5.0),
        # target geometry (relative only)
        "target_depth_yd":         target_depth,          # |QB -> target|
        "margin_to_target":   receiver_to_target - def1_to_target,    
        "man_cov_flag" : encode_man_zone(play_row.get("team_coverage_man_zone", np.nan)),
        **route_oh,

    }


In [17]:
# dfPlays must have: week, game_id, play_id
plays_sorted = dfPlays.sort_values(["week","game_id","play_id"]).reset_index(drop=True)
all_feats = []
for week in plays_sorted["week"].dropna().astype(int).unique():
    print(f'Current week: {week}')
    trk_w = load_week(week)
    if trk_w.empty:
        print("Nothing found")
        continue

    wk_plays = plays_sorted[plays_sorted["week"].astype(int).eq(week)]
    for _, prow in wk_plays.iterrows():
        gid, pid = int(prow["game_id"]), int(prow["play_id"])
        trk_play = trk_w[(trk_w["game_id"]==gid) & (trk_w["play_id"]==pid)]
        if trk_play.empty:
            continue

        f_throw = get_throw_frame_last(trk_play)
        if f_throw is None:
            continue

        frame0 = trk_play[trk_play["frame_id"]==f_throw]
        if frame0.empty:
            continue

        feat = features_at_throw_lastframe(frame0, prow)
        if feat:
            all_feats.append(feat)


features_df = pd.DataFrame(all_feats)
len(features_df)


Current week: 1
Current week: 2
Current week: 3
Current week: 4
Current week: 5
Current week: 6
Current week: 7
Current week: 8
Current week: 9
Current week: 10
Current week: 11
Current week: 12
Current week: 13
Current week: 14
Current week: 15
Current week: 16
Current week: 17
Current week: 18


8924

In [18]:
# after: features_df = pd.DataFrame(all_feats)

# Build binary label: 1 if Complete ('C'), else 0
lbl = dfPlays[["game_id","play_id","pass_result"]].copy()
lbl["y_complete"] = (lbl["pass_result"].astype(str).str.upper() == "C").astype(int)

# (optional) keep only rows where we actually know the outcome
lbl = lbl[lbl["pass_result"].notna()]

# Merge onto features and keep the label only
dataset = features_df.merge(lbl[["game_id","play_id","y_complete"]],
                            on=["game_id","play_id"], how="inner")

# (optional) tidy & save
dataset.to_parquet(f"{projectRoot}/features/v2/dataset_at_throw.parquet", index=False)


OSError: Cannot save file into a non-existent directory: '/Users/luisbapst/Code/BDB26/features/features/v2'